# Position2Go keypoint localization network


In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras as keras
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *

import sys, os, warnings, time, glob

# Helper libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print("keras version {}".format(tf.keras.__version__)); del keras
print("tensorflow version {}".format(tf.__version__))

from keypoints_localizer.tf_records_generator import *

%load_ext autoreload
%autoreload 2

keras version 2.2.4-tf
tensorflow version 1.13.1


## Define data loader

In [2]:
# Define global variables
EPOCHS = 30
SUM_OF_ALL_DATASAMPLES = 200
BATCHSIZE = 8
SHUFFLE_BUFFER = 256

In [3]:
TEST_SET_PATH = "/home/kingkolibri/10_catkin_ws/test_records/"
TRAIN_SET_PATH = "/home/kingkolibri/10_catkin_ws/train_records/"

In [4]:
train_files = glob.glob(TRAIN_SET_PATH + "p2g_*.tfrecord")
test_files = glob.glob(TEST_SET_PATH + "p2g_*.tfrecord")
train_files

['/home/kingkolibri/10_catkin_ws/train_records/p2g_20190823-bauingenieurwesen.tfrecord']

In [5]:
def create_dataset(filenames, repetitions=-1):
    
    # This works with arrays as well
    dataset = tf.data.TFRecordDataset(filenames=filenames)
    
    # Maps the parser on every filepath in the array
    dataset = dataset.map(parse_p2g_example, num_parallel_calls=8)
    dataset = dataset.repeat(repetitions) # will go on forever        
    dataset = dataset.shuffle(SHUFFLE_BUFFER)
    dataset = dataset.batch(BATCHSIZE)
    
    # Create an iterator
    iterator = dataset.make_one_shot_iterator()
    
    # Create tf representation of the iterator
    raw, _, heatmaps = iterator.get_next()
    
    return raw, heatmaps

## Build and train model

In [6]:
STEPS_PER_EPOCH= int(SUM_OF_ALL_DATASAMPLES / BATCHSIZE)

#Get your datatensors
inputs, labels = create_dataset(train_files)

# Define Input
model_input = tf.keras.layers.Input(tensor=inputs)

# Encdoing layer
x = model_input
for i in range(0,5):
    x = Conv2D(filters=8 + 8*i,
               kernel_size=[5, 5],
               data_format='channels_last',
               padding="same")(x)
    x = BatchNormalization()(x, training=True)
    x =  Activation('relu')(x)
    
    x = Conv2D(filters=8 + 8*i,
           kernel_size=[5, 5],
           strides=[2, 1],
           data_format='channels_last',
           padding="same")(x)
    x = BatchNormalization()(x, training=True)
    x = Activation('relu')(x)
    
# Decoding layer
for i in range(0,3):
    x = Conv2DTranspose(filters=32,
                kernel_size=[6, 6],
                strides=[2, 1],
                padding="same")(x)
    x = Activation('relu')(x)
       
x = Conv2DTranspose(filters=13,
            kernel_size=[4, 4],
            strides=[2, 2],
            padding="same")(x)
x = Activation('sigmoid')(x)

# model_output = tf.keras.layers.Dense(32*32*13, activation='relu')(model_output)

#Create your model
localizer_model = tf.keras.models.Model(inputs=model_input, 
                                 outputs=x
                                )


# Describe model
localizer_model.summary()

ValueError: Input 0 of layer conv2d is incompatible with the layer: expected ndim=4, found ndim=5. Full shape received: [None, 2, 256, 64, 8]

### Train model

In [46]:
#Compile model
localizer_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.01),
                        loss='mean_squared_error',
                        metrics=['mean_squared_error', 'acc'],
                        target_tensors=[labels]
                   )

In [47]:
64/4

16.0

In [48]:
#Train the model
localizer_model.fit(epochs=EPOCHS,
                    steps_per_epoch=STEPS_PER_EPOCH)

Epoch 1/30
25/25 [==============================] - 7s 263ms/step - loss: 271.7155 - mean_squared_error: 271.7155 - acc: 0.1427
Epoch 2/30
25/25 [==============================] - 3s 136ms/step - loss: 304.0161 - mean_squared_error: 304.0161 - acc: 0.3295
Epoch 3/30
25/25 [==============================] - 4s 143ms/step - loss: 296.1530 - mean_squared_error: 296.1530 - acc: 0.5503
Epoch 4/30
25/25 [==============================] - 4s 143ms/step - loss: 290.9292 - mean_squared_error: 290.9292 - acc: 0.3239
Epoch 5/30
25/25 [==============================] - 4s 144ms/step - loss: 302.6887 - mean_squared_error: 302.6887 - acc: 0.2628
Epoch 6/30
25/25 [==============================] - 4s 146ms/step - loss: 269.0944 - mean_squared_error: 269.0944 - acc: 0.2715
Epoch 7/30
25/25 [==============================] - 4s 145ms/step - loss: 303.4221 - mean_squared_error: 303.4222 - acc: 0.3123
Epoch 8/30
25/25 [==============================] - 4s 144ms/step - loss: 295.9064 - mean_squared_error:

### Test trained model

In [58]:
def create_dataset(filenames, repetitions=-1):
    
    # This works with arrays as well
    dataset = tf.data.TFRecordDataset(filenames=filenames)
    
    # Maps the parser on every filepath in the array
    dataset = dataset.map(parse_p2g_example, num_parallel_calls=8)
    dataset = dataset.repeat(repetitions) # will go on forever        
    dataset = dataset.shuffle(SHUFFLE_BUFFER)
    dataset = dataset.batch(BATCHSIZE)
    
    # Create an iterator
    iterator = dataset.make_one_shot_iterator()
    
    # Create tf representation of the iterator
    _, rdm, heatmaps = iterator.get_next()
    
    return rdm, heatmaps

In [61]:
#Get your datatensors
inputs, labels = create_dataset(test_files, repetitions=1)

#Combine it with keras
model_input = tf.keras.layers.Input(tensor=inputs)

localizer_model.evaluate(
    x=test_set_iterator,
    verbose=1,
)


TypeError: The model has multiple outputs, so `sample_weight` should be either a list or a dict. Provided `sample_weight` type not understood: Tensor("IteratorGetNext_13:2", shape=(?, 128, 128, 13), dtype=float32)

## Visualize result

In [53]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables()) #execute init_op
    sample_rdm = sess.run(inputs)
    sample_heatmaps = sess.run(labels)
    sample_outputs = localizer_model.predict_on_batch(inputs)

OutOfRangeError: End of sequence
	 [[node IteratorGetNext_7 (defined at <ipython-input-44-f3aeca157c14>:16) ]]

Caused by op 'IteratorGetNext_7', defined at:
  File "/home/kingkolibri/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/kingkolibri/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/kingkolibri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/kingkolibri/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/kingkolibri/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/kingkolibri/anaconda3/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "/home/kingkolibri/anaconda3/lib/python3

In [ ]:
sample_outputs.shape

### Visualize sample heatmaps

In [ ]:
keypoint_columns = [
    'LAnkle', 
    'LElbow', 
    'LHip', 
    'LKnee',
    'LShoulder',
    #['keypoint_0_LSmallToe_x', 'keypoint_0_LSmallToe_y'],
    'LWrist',
    #['keypoint_0_MidHip_x', 'keypoint_0_MidHip_y'],
    #['keypoint_0_Neck_x', 'keypoint_0_Neck_y'],
    'Nose',
    'RAnkle',
    #['keypoint_0_RBigToe_x','keypoint_0_RBigToe_y'], 
    #['keypoint_0_REar_x','keypoint_0_REar_y'], 
    'RElbow', 
    #['keypoint_0_REye_x','keypoint_0_REye_y'], 
    #['keypoint_0_RHeel_x','keypoint_0_RHeel_y'], 
    'RHip', 
    'RKnee', 
    'RShoulder',
    #['keypoint_0_RSmallToe_x', 'keypoint_0_RSmallToe_y'], 
    'RWrist' 
]

### Visualize estimated keypoints

In [ ]:
y_test = sample_outputs
y_pred = sample_heatmaps

In [ ]:
heigth = y_test.shape[1]
width = y_test.shape[2]
num_keypoints = y_test.shape[3]



for i in np.random.randint(y_test.shape[0], size=(2,)):
    
    
    fig = plt.figure(figsize=(3,3))
  #  ax = fig.add_subplot(1,1,1)
  #  ax.imshow(X_train[i,:,:,0], cmap="gray")
  #  ax.axis("off")
    
    fig = plt.figure(figsize=(20,3))
    count = 1
    
    for j, keypoint in enumerate(keypoint_columns):
        ax = fig.add_subplot(2, num_keypoints, count)
        ax.set_title(keypoint)
        ax.axis("off")
        count += 1
        ax.imshow(y_pred[i,:,:,j])
        if j == 0:
            ax.set_ylabel("prediction")
            
    for j, keypoint in enumerate(keypoint_columns):
        ax = fig.add_subplot(2, num_keypoints,count)
        count += 1
        ax.imshow(y_test[i,:,:,j])   
        ax.axis("off")
        if j == 0:
            ax.set_ylabel("true")
    plt.show()

### Visualize stick man

In [ ]:
keypoint_pairs = [
    [('Neck', 'RShoulder'), None],
    [('Neck', 'LShoulder'), None],
    [('RShoulder', 'RElbow'), None],
    [('RElbow', 'RWrist'), None],
    [('LShoulder', 'LElbow'), None],
    [('LElbow', 'LWrist'), None],
    [('Neck', 'RHip'), None],
    [('RHip', 'RKnee'), None],
    [('RKnee', 'RAnkle'), None],
    [('Neck', 'LHip'), None],
    [('LHip', 'LKnee'), None],
    [('LKnee', 'LAnkle'), None],
    [('Neck', 'Nose'), None],
    [('Nose', 'REye'), None],
    [('REye', 'REar'), None],
    [('Nose', 'LEye'), None],
    [('LEye', 'LEar'), None],
    [('RShoulder', 'REar'), None],
    [('LShoulder', 'LEar'), None],
    [('LHip', 'LAnkle'), 'LKnee'],
    [('RHip', 'RAnkle'), 'RKnee'],
    [('RShoulder', 'RAnkle'), 'RElbow'],
    [('LShoulder', 'LAnkle'), 'LElbow'],
    [('RHip', 'LHip'), 'Neck'],
    [('RHip', 'RShoulder'), 'Neck'],
    [('LHip', 'LShoulder'), 'Neck'],
]

In [ ]:
body_pred = {}
body_true = {}
for j, keypoint in enumerate(keypoint_columns):
    x_pred = y_pred[i,:,:,j]
    x_true = y_test[i,:,:,j]
    body_pred[keypoint] = np.unravel_index(y_test[i,:,:,j].argmax(), y_test[i,:,:,j].shape)
    body_true[keypoint] = np.unravel_index(y_pred[i,:,:,j].argmax(), y_pred[i,:,:,j].shape)

Let's first check the visualization of the samples

In [ ]:
body_true

In [ ]:
img = np.zeros((64,64,3), np.uint8)

import cv2
image_h, image_w = img.shape[:2]

centers = {}

# draw point
for key in keypoint_columns:
    if key not in body_true.keys():
        continue

    body_part = body_true[key]
    center = (body_part[1], body_part[0])
    centers[key] = center
    img[center[1], center[0], :] =  [0, 255, 0]

In [ ]:
plt.imshow(img)

In [ ]:
# draw line
for pair_order, pair in enumerate(keypoint_pairs):
    if pair[0][0] not in body_true.keys() or pair[0][1] not in body_true.keys() or pair[1] in body_true.keys():
        continue

    cv2.line(img, centers[pair[0][0]], centers[pair[0][1]], [255, 0, 0], 1)

In [ ]:
plt.imshow(img)

Now lets take a look at the network output

In [ ]:
img = np.zeros((64,64,3), np.uint8)

import cv2
image_h, image_w = img.shape[:2]

centers = {}

# draw point
for key in keypoint_columns:
    if key not in body_pred.keys():
        continue

    body_part = body_pred[key]
    center = (body_part[1], body_part[0])
    centers[key] = center
    img[center[1], center[0], :] =  [0, 255, 0]
    
# draw line
for pair_order, pair in enumerate(keypoint_pairs):
    if pair[0][0] not in body_pred.keys() or pair[0][1] not in body_pred.keys() or pair[1] in body_pred.keys():
        continue

    cv2.line(img, centers[pair[0][0]], centers[pair[0][1]], [255, 0, 0], 1)
    
plt.imshow(img)